
# 📱 Estudo de Caso – Detecção de SPAM em SMS (Classificação de Texto)
**Professor Rodrigo – UniFECAF** • **Duração:** ~2h • **Trabalho:** dupla/trio

## 🎯 Objetivo
Construir um pipeline de **Machine Learning para NLP clássico** que classifica mensagens SMS como **SPAM** ou **HAM (legítima)**.

- Revisar o **pipeline** de ML (Aulas 01–02).
- Praticar **pré-processamento** de texto e **vetorização** (TF-IDF).
- Treinar e avaliar modelos (Naive Bayes, Regressão Logística).
- Métricas: **Accuracy, Precision, Recall, F1, ROC-AUC** e **RMSE educacional** (probabilidade vs rótulo).



## 📂 Dataset (download e upload manual)
Usaremos o **SMS Spam Collection** (UCI Repository).

🔗 Baixe pelo site oficial da UCI:  
https://archive.ics.uci.edu/dataset/228/sms+spam+collection

> O arquivo costuma se chamar **`SMSSpamCollection`** (texto separado por **TAB**, duas colunas: `label` e `text`).  
> Depois de baixar, **faça o upload** do arquivo na célula abaixo.


In [ ]:

# 📥 Upload do arquivo da UCI (ex.: SMSSpamCollection)
from google.colab import files
import io, pandas as pd

print("Selecione o arquivo 'SMSSpamCollection' baixado da UCI")
uploaded = files.upload()

filename = next(iter(uploaded))

# Tenta ler com header inexistente (duas colunas: label, text)
try:
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep='\t', header=None, names=['label','text'], encoding='utf-8')
except Exception as e:
    print("Falha no parse padrão (tab). Tentando outras variações...", e)
    df = pd.read_csv(io.BytesIO(uploaded[filename]), sep='\t', encoding_errors='ignore', header=None, names=['label','text'])

print("Amostra:")
df.head()



## 1️⃣ Revisão rápida – Pipeline de ML
1. Coleta de dados → 2. EDA → 3. Pré-processamento → 4. Treinamento → 5. Avaliação → 6. Recomendação

**Pergunta (responda em texto):** Em qual etapa você imagina que gastaremos mais tempo hoje e por quê?


*Escreva sua resposta aqui*


## 2️⃣ EDA – Exploração de Dados
Vamos entender a base: quantidade, balanceamento, tamanho dos textos.


In [ ]:

# Informações gerais
df.info()


In [ ]:

# Distribuição de classes
prop = df['label'].value_counts(normalize=True)
prop


In [ ]:

# Tamanho dos textos (número de caracteres) - visão rápida
import matplotlib.pyplot as plt

df['len'] = df['text'].astype(str).str.len()
print(df['len'].describe())
plt.figure()
df['len'].hist(bins=30)
plt.title("Distribuição do tamanho das mensagens (caracteres)")
plt.xlabel("nº de caracteres"); plt.ylabel("freq.")
plt.show()



**Perguntas rápidas:**
1. A base é balanceada? O que isso significa para **accuracy**?  
2. Mensagens de spam tendem a ser maiores ou menores?


*Responda aqui*


## 3️⃣ Pré-processamento de Texto
- Limpeza simples (lowercase).  
- **TF-IDF** para transformar texto em vetores numéricos.  
- **Split** treino/teste com estratificação.


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Label -> binária (spam=1, ham=0)
df['label_bin'] = (df['label'].str.lower().str.strip() == 'spam').astype(int)

X_text = df['text'].astype(str)
y = df['label_bin']

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.30, random_state=42, stratify=y
)

# Vetorização TF-IDF (limpeza básica embutida: lowercase=True)
tfidf = TfidfVectorizer(lowercase=True, stop_words=None)  # pode testar stop_words='english'
X_train = tfidf.fit_transform(X_train_text)
X_test  = tfidf.transform(X_test_text)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()



## 4️⃣ Modelos e Métricas
Vamos comparar **Naive Bayes (MultinomialNB)** e **Regressão Logística**.

**Métricas:**  
- **Accuracy**: % de acertos.  
- **Precision**: dos que o modelo chamou de *spam*, quantos eram *spam*.  
- **Recall**: dos *spam* reais, quantos o modelo pegou.  
- **F1**: equilíbrio entre precision e recall.  
- **ROC-AUC**: quão bem o modelo separa classes variando o limiar.  
- **RMSE (educacional)**: erro médio das **probabilidades** previstas vs rótulos (0/1).


In [ ]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, classification_report,
    roc_auc_score, RocCurveDisplay
)
import numpy as np
import matplotlib.pyplot as plt

models = {
    "MultinomialNB": MultinomialNB(),
    "LogReg": LogisticRegression(max_iter=200, n_jobs=-1, random_state=42)
}

for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, digits=4))
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.title(f"{name} - Matriz de Confusão (Teste)")
    plt.show()

    # Probabilidades para AUC e RMSE (se disponível)
    if hasattr(model, "predict_proba"):
        y_score = model.predict_proba(X_test)[:,1]
    else:
        # fallback para modelos sem predict_proba (não é o caso aqui)
        y_score = None

    if y_score is not None:
        auc = roc_auc_score(y_test, y_score)
        print(f"ROC-AUC: {auc:.4f}")
        RocCurveDisplay.from_predictions(y_test, y_score)
        plt.title(f"{name} - Curva ROC (Teste)")
        plt.show()

        # RMSE educacional (probabilidade vs rótulo)
        rmse = np.sqrt(np.mean((y_score - y_test.values)**2))
        print(f"RMSE (probabilidade vs rótulo): {rmse:.4f}")



### 💬 Perguntas finais (responda em texto)
1. Qual modelo apresentou melhor equilíbrio entre **precision** e **recall**?  
2. Para um filtro de spam corporativo, você priorizaria **precision** ou **recall**? Explique.


*Escreva aqui suas respostas*


## 📝 Reflexão do Grupo (obrigatória)
**Dificuldades encontradas:**  
- ...

**O que aprendemos:**  
- ...

**Como aplicar na TI (segurança, e-mail corporativo, antiphishing):**  
- ...
